# Topic Annotation

In [2]:
%load_ext autoreload
%autoreload 2

In [16]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import topics
import pandas as pd
meta = pd.read_csv('bill-meta.csv', index_col='id')
meta[['title']].head(3)

,title
id,
115 H.R. 3823,Disaster Tax Relief and Airport and Airway Ext...
115 H.R. 88,Shiloh National Military Park Boundary Adjustm...
115 H.R. 1,An Act to provide for reconciliation pursuant ...


In [4]:
dictionary = Dictionary.load('data/bills_prune.dict')
lda_40 = LdaModel.load('data/lda_40.gensim')
lda_40._dictionary = dictionary

In [14]:
from SRP import Vector_file
with Vector_file('data/topic-dists-title.bin') as vfile:
    allbills = vfile.to_matrix()
allbills['matrix'].shape

(5531, 40)

In [17]:
annotations = [['Appropriations', 'Yes'], ['Family', 'Yes'], ['???', 'No'], ['???', 'No'], ['???', 'Maybe'], ['Land Use', 'Yes'], ['???', 'No'], 
               ['Catch-all / Native Nations, Water', 'No'], ['Health Planning', 'Yes'], ['History/Memorial', 'Yes'], ['Corporations', 'Yes'], ['Forests', 'Yes'], ['Service/Function', 'No'], ['???', 'No'], ['Transactions', 'Yes'], ['Revenue', 'No'], ['Banking Regulation', 'Yes'], ['Employment', 'Yes'], ['Disability', 'Yes'], ['Compensation', 'Yes'], ['Sections', 'No'], ['Indigenous Territory', 'Yes'], ['Aerospace and Defense', 'Yes'], ['Bio/Chem', 'Yes'], ['Paragraphs', 'No'], ['Retirement', 'Yes'], ['Ports and Place', 'Yes'], ['???', 'No'], ['Leave and Relief', 'Yes'], ['Oversight', 'Maybe'], ['Water', 'Yes'], ['Land', 'Yes'], ['Fraud', 'Yes'], ['Judiciary', 'Yes'], ['???', 'No'], ['???', 'No'], ['Property and Compensation', 'Yes'], ['???', 'No'], ['???', 'No'], ['???', 'No']]
annotations = pd.DataFrame(annotations, columns=['category', 'useful'])

tr = topics.TopicReport(lda_40)
top_words = pd.Series([" ".join(tr.top_words(topicno)) for topicno in range(tr.lda.num_topics)])
annotations['words'] = top_words
annotations.query('useful == "Yes"').sample(5)

,category,useful,words
16,Banking Regulation,Yes,board modify preservation bank judge ongoing r...
32,Fraud,Yes,prevent comptroller critical guideline missour...
17,Employment,Yes,employment market industry particularly reflec...
22,Aerospace and Defense,Yes,space meeting make write defense terminate exp...
23,Bio/Chem,Yes,remain obligation loss eligibility device resp...


In [18]:
# Get top bills for a given topic number
#topicn += 1
topicn = 6
top_probs = tr.top_docs(topicn, allbills['matrix'], allbills['names'])
meta.loc[top_probs, 'title'].tolist()

['To extend the authorization of the Drug-Free Communities Support Program for an additional 5 years, to authorize a National Community Antidrug Coalition Institute, and for other purposes.',
 'Seneca Nation Settlement Act of 1990',
 'Judicial Amendments Act of 1994',
 'Drug-Free Communities Act of 1997',
 'Marrakesh Treaty Implementation Act',
 'Fort McDowell Indian Community Water Rights Settlement Act of 2006',
 'To amend section 326 of the Higher Education Act of 1965 to permit continued participation by Historically Black Graduate Professional Schools in the grant program authorized by that section.',
 'To allow for election of the Delegate from Guam by other than separate ballot, and for other purposes.',
 'A bill to disapprove of amendments to the Federal Sentencing Guidelines relating to lowering of crack sentences and sentences for money laundering and transactions in property derived from unlawful activity.',
 'To amend the Act of October 15, 1966 (80 Stat. 915), as amended, 

In [19]:
annotations.to_csv('data/lda_40.annotations.gz', compression='gzip')